In [943]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [944]:
import json
from inspect import signature

In [945]:
from ollama import Client
from IPython.display import display_markdown, display_latex

client = Client(host="http://localhost:11434")

model_name = "qwen2.5:7b"

# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

### Auxiliar functions

In [946]:
def parse_multiple_tool_calls(tool_call_str: str) -> list:
    """
    Parse multiple tool_call XML entries and return a list of corresponding dictionaries.

    Parameters
    ----------
    tool_call_str : str
        A string containing multiple XML tool calls.

    Returns
    -------
    list
        A list of dictionaries containing the parsed JSON from each tool_call tag.
    """
    # Parse the XML string
    root = ET.fromstring(f"<root>{tool_call_str}</root>")  # Wrap in a root for valid XML

    # Iterate over all <tool_call> elements and extract their JSON content
    tool_calls = []
    for tool_call in root.findall("tool_call"):
        json_content = tool_call.text.strip()
        tool_call_dict = json.loads(json_content)
        tool_calls.append(tool_call_dict)

    return tool_calls


def process_tool_calls(tool_call_str):
    """
    Process a string of tool calls and return the results formatted in XML-like tags.

    This function takes a string containing multiple tool calls, parses each call to 
    identify the tool name and its arguments, executes the corresponding function 
    from the tool registry, and collects the results.

    Parameters:
    -----------
    tool_call_str : str
        A string containing multiple tool call segments formatted as 
        '<tool_call>...</tool_call>', where each segment contains JSON data 
        specifying the tool name, arguments, and a unique identifier.

    Returns:
    --------
    str
        A string that contains the results of the executed tool calls, 
        formatted within '<tool_response>' tags. Each result corresponds 
        to the respective tool call and is presented in the order they were 
        processed.
    """
    tool_call_result = []
    for tool_info in parse_multiple_tool_calls(tool_call_str):
        # Retrieve the function based on tool name and execute it with provided arguments
        values = {
            "id": tool_info['id'],
            "result": tool_registry.get_function(tool_info['name'])(**tool_info['arguments'])
        }
        tool_call_result.append(values)

    # Format the results into XML-like output
    output = ''.join(f'<tool_response>\n{item["result"]}\n</tool_response>\n' for item in tool_call_result)
    return output


### Tools Registry

In [947]:
class ToolRegistry:
    """
    A class to manage and register available tools (functions).
    
    This registry allows the user to register tools, retrieve them, and generate
    JSON schemas for each tool's function signature.
    """

    def __init__(self):
        """
        Initialize a ToolRegistry instance.
        
        Attributes
        ----------
        tools : dict
            A dictionary that stores tools with their names as keys and functions as values.
        """
        self.tools = {}

    def register(self, name: str, func):
        """
        Register a new function as a tool.
        
        Parameters
        ----------
        name : str
            The name of the tool to be registered.
        func : function
            The function to register as a tool.
        
        Raises
        ------
        TypeError
            If `func` is not callable.
        """
        if not callable(func):
            raise TypeError(f"The tool '{name}' must be a callable function.")
        self.tools[name] = func

    def get_function(self, name: str):
        """
        Get a registered function by its name.
        
        Parameters
        ----------
        name : str
            The name of the tool (function) to retrieve.
        
        Returns
        -------
        function or None
            The function registered under the given name, or None if not found.
        """
        return self.tools.get(name)

    def get_json_schema(self, func):
        """
        Generate the JSON schema for a given function.
        
        Parameters
        ----------
        func : function
            The function for which to generate the JSON schema.
        
        Returns
        -------
        str
            A JSON-formatted string representing the function's signature schema.
        
        Raises
        ------
        ValueError
            If the provided object is not a callable function.
        """
        if not callable(func):
            raise ValueError("The argument must be a callable function.")
        
        sigs = signature(func)
        desc = func.__doc__ or "No description provided."
        name = func.__name__

        schema = {
            "name": name,
            "description": desc,
            "parameters": {
                "properties": {
                    param_name: {"type": param.annotation.__name__ if param.annotation != param.empty else "unknown"}
                    for param_name, param in sigs.parameters.items()
                }
            }
        }

        return json.dumps(schema, indent=4)

    def generate_tools_schema(self):
        """
        Generate JSON schemas for all registered tools.
        
        Returns
        -------
        str
            A concatenation of JSON schemas for all tools, each schema on a new line.
        """
        schemas = [self.get_json_schema(func) for func in self.tools.values()]
        return "\n".join(schemas)

### Define Tools

In [948]:

def summation(x: int, y: int = 0) -> int:
    """
    Add two integers.

    Parameters
    ----------
    x : int
        The first integer to be added.
    y : int, optional
        The second integer to be added (default is 0).

    Returns
    -------
    int
        The sum of `x` and `y`.
    """
    return x + y
    

def multiplication(x: int, y: int) -> int:
    """
    Multiply two integers.

    Parameters
    ----------
    x : int
        The first integer to be multiplied.
    y : int
        The second integer to be multiplied.

    Returns
    -------
    int
        The product of `x` and `y`.
    """
    return x * y

In [949]:
# Instanciar o registro de ferramentas
tool_registry = ToolRegistry()

# Registrar as funções
tool_registry.register("summation", summation)
tool_registry.register("multiplication", multiplication)

tools_schemas_string = tool_registry.generate_tools_schema()

In [953]:
TOOL_PROMPT = """You are a function calling AI model.
You may call one or more functions to assist with the user query. If a function or tool is not available, use training data to respond. You are provided with function signatures within <tools></tools> XML tags:
For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:

Intruction:
    - Aways verify if the tools outputs need to be combined or not.
    - Avoid LaTex.

<tool_call>
{"name": <function-name>, "arguments": <args-json-object>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools:

<tools>
%s
</tools>
""" % tools_schemas_string

In [956]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_PROMPT,
    }
]

agent_chat_history = []

USER_PROMPT = "Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?"

tool_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)
agent_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)

# Request tools action
tools_calls_request = client.chat(model=model_name, messages=tool_chat_history)

# Parse the response into model default format
tools_calls_str = tools_calls_request['message']['content']
output = process_tool_call(tools_calls_str)

# Return the final response using the tools retrivied info
agent_chat_history.append({
    "role": "user",
    "content": output
})

tool_response = client.chat(model=model_name, messages=agent_chat_history)

response_using_tool = tool_response['message']['content']

In [957]:
display_markdown(response_using_tool, raw=True)

Sure! Here are the results:

- The product of 25 multiplied by 12 is 300.
- The sum of 3 and 6 is 9.

Regarding your request to search what an LLM is, while I don't perform web searches directly, I can tell you that LLM stands for Large Language Model. It's a type of artificial intelligence model designed to understand and generate human-like text. These models are often used in applications like language translation, chatbots, and content generation.